In [5]:
import pandas as pd
import numpy as np
import datasets
from collections import Counter
from transformers import AutoTokenizer

import pandas as pd
from datasets import load_dataset,Dataset,concatenate_datasets
import jsonlines
import json

In [11]:
translation_template='''You are a professional translator. Translate the following {} source text into {}. Refer to the word pairs in the glossary if the glossary exists when you translate. Do not translate the glossary it self.
{}
Translation:
'''
response_template="Translation:"
glossary_template="Glossary:"
sentence_template="Source:"

In [23]:
import os,sys
import random
import numpy as np
import torch
import pandas as pd
from datasets import load_dataset,Dataset,concatenate_datasets
import jsonlines
import ipdb
import json
from kiwipiepy import Kiwi
from nltk.tokenize import sent_tokenize
import ast
import ipdb

kiwi = Kiwi()

def formatting_prompt_func(template:str,*args:str):
  '''
  Template can be like ;
  ### Input :
  {}
  ### Output :
  {}
  '''

  return template.format(*args)


def make_translation_input_from_dataset(data,
                                  tokenizer,
                                  prompt_template,
                                  glossary_template=None,
                                  sentence_template=None,
                                  src:str=None, 
                                  tgt:str=None,
                                  return_output=True,
                                  text_split=True,
                                  **kwargs
                                  ):

    lang_dict={"korean":"Korean","english":"English","ko":"Korean","eng":"English","en":"English"}
    src_tgt_dict={"en":"english","eng":"english","english":"english","ko":"korean","kor":"korean","korean":"korean"}

    if not src and not tgt:
        if "src" in data.keys() and "tgt" in data.keys():
            src=src_tgt_dict[data["src"]]
            tgt=src_tgt_dict[data["tgt"]]
        else:
            raise Exception("'src'와 'tgt'가 주어지거나, data의 key로 존재해야합니다.")

    src_text=data[src]
    formatted_text=None

    if text_split:
        splited_sents=split_sents(lang_dict[src],src_text)
        sent2terms = []
        
        if data["term_dict"] is not None and len(data["term_dict"]):
            term_dict = ast.literal_eval(data["term_dict"])

            for s in splited_sents:
                new_sent_parts = {}
                for k, v in term_dict.items():
                    if k in s:
                        new_sent_parts[k]=v

                if len(new_sent_parts):
                    new_sent_parts=formatting_glossary(new_sent_parts,glossary_template)
                    new_s = f"{sentence_template}\n{s}\n{new_sent_parts}\n"
                else:
                    new_s=f"{sentence_template}\n{s}\n"

                sent2terms.append(new_s)
        else:
            # Handle case of empty term_dict (e.g., directly append sentences)
            for s in splited_sents:
                new_s = f"{sentence_template}\n{s}\n"
                sent2terms.append(new_s)

        formatted_text="".join(sent2terms).rstrip()
            

    else:
        if data["term_dict"] is not None and len(data["term_dict"]):
            term_dict = ast.literal_eval(data["term_dict"])
            term_dict=formatting_glossary(term_dict,glossary_template)
            formatted_text=f"{sentence_template}\n{src_text}\n{term_dict}"

    template=formatting_prompt_func(prompt_template,lang_dict[src],lang_dict[tgt],formatted_text)

    if return_output:
        template=template+data[tgt]+tokenizer.eos_token
        
    return {"text":template}


def formatting_glossary(term_dict,glossary_template):
    glossary=[f"{k}={v}" for k,v in term_dict.items()]
    glossary_str="\n".join(glossary)
    glossary_str=f"{glossary_template}\n{glossary_str}".strip()

    return glossary_str

def split_sents(lang,
                text,
                ):

    lang_dict={"korean":"korean","ko":"korean","kor":"korean","eng":"english","english":"english","en":"english"}
    src = lang_dict[lang.lower()]
    
    splited_sents=[]
    paras=text.split("\n") #split text into paragraphs based on linebreak to keep its original format.
    
    for idx,para in enumerate(paras):
        if len(para.strip()):
            if src=="korean":
                temp_sents=[s.text for s in kiwi.split_into_sents(para)]
            else:
                temp_sents=sent_tokenize(para)
            if idx<len(paras)-1:
                temp_sents[-1]+="\n" #keep linebreak
            splited_sents.extend(temp_sents)
        else:
            splited_sents[-1]+="\n"

    return splited_sents
            

In [4]:
train_dataset=Dataset.load_from_disk("/azurestorage/data/translation_data/aligned_dataset/prepared_for_training/training_dataset_10k/")

In [10]:
tokenizer=AutoTokenizer.from_pretrained("/azurestorage/models/production/tmt-eeve16k/")

In [39]:
result=make_translation_input_from_dataset(train_dataset[9],
                                  tokenizer,
                                  prompt_template=translation_template,
                                  glossary_template=glossary_template,
                                  sentence_template=sentence_template,
                                  return_output=True,
                                  text_split=False,
                                  )

print(result["text"])

You are a professional translator. Translate the following English source text into Korean. Refer to the word pairs in the glossary if the glossary exists when you translate. Do not translate the glossary it self.
Source:
Article 48-2 (Inspections of Large Shareholders)(1)Where any of the following persons (hereafter referred to as "large shareholder, etc." in this Article) fall under any of the following items, the Financial Services Commission may have the Governor of the Financial Supervisory Service inspect business and asset conditions of the relevant large shareholder, etc. within the minimum extent necessary for the purposes thereof: <Amended on Aug. 13, 2013>1.A person subject to conversion:(a)Where necessary to check the inspection result underArticle 16-3(2);(b)Where it is acknowledged that a person subject to conversion will be highly likely to engage in illegal transactions with a bank due to unhealthy financial status, such as rapid increase in debts and occurrence of huge

# New train dataset

In [30]:
train_dataset=datasets.Dataset.load_from_disk("/azurestorage/data/translation_data/alinged_dataset/prepared_for_training/training_dataset_20k/")

In [85]:
train_df=train_dataset.to_pandas()

In [100]:
train_df_under_100=train_df[train_df["korean"].str.len()<100].sample(frac=0.35)
train_df_over_100=train_df[train_df["korean"].str.len()>=100]

In [104]:
train_df_sample_10k=pd.concat([train_df_under_100,train_df_over_100]).sample(frac=1)
train_df_sample_10k.head(5)

,english,korean,category,term_dict,id,src,tgt,length_group
19010,Inigmaville with area for exclusive use of 244...,서울 강남구 청담동 이니그마빌 전용 244.77㎡도 2014년 10월 32억 원에 ...,경제,{'서울 강남구 청담동 이니그마빌': 'Inigmaville in Cheongdam...,618344.0,korean,english,2.0
1927,As Icarus does not itself contain a genetic ma...,Icarus 자체에는 유전자 표지자가 포함되어 있지 않기 때문에 snw 검사를 통해...,food,,1336861.0,english,korean,NaN
6779,"Therefore, the method of obtaining ETR can be ...",이에 ETR을 구하는 방법은 크게 GAAP ETR과 Cash ETR로 나눌 수 있으...,법률,"{'ETR': 'ETR', 'GAAP ETR': 'GAAP ETR', 'Cash E...",1249742.0,english,korean,NaN
9508,"Consequently, in this study, element decomposi...",이에 본 연구에서는 1997∼2008년과 2008∼2011년의 두 기간으로 구분하여...,경제,,1265691.0,english,korean,NaN
15476,"In France, when signing a legal document, it i...","프랑스에서는 통상적으로 법적 문서에 서명할 경우, 서명에 앞서 ""읽고 동의함""(lu...",법률,,1274058.0,english,korean,NaN


# Add tax dictionary & tax paper pairs

In [13]:
dict_df=dict_df.drop_duplicates(subset=[0,1])

In [20]:
dict_df=dict_df.drop_duplicates(subset=[1])

In [113]:
dict_df=dict_df.sample(frac=1)[[0,1]]

In [114]:
dict_df.columns=["korean","english"]

In [116]:
tax_law=pd.read_excel("/azurestorage/data/translation_data/raw_data/세법논문_pairing.xlsx")[["Korean","English"]]
tax_law=tax_law.dropna(axis=0)
tax_law.columns=["korean","english"]

In [118]:
idx=34

print(tax_law["korean"].iloc[idx])
print("-----------------------------")
print(tax_law["english"].iloc[idx])

많은 다국적기업이 해외 시장을 개척하거나 시장점유율을 높이는 데 소요되는 광고선전비를 유통기능을 담당하는 각국의 자회사에 부담시키고, 나아가 그룹 차원의 국제마케팅활동에 소요된 비용을 각국의 자회사들로부터 국제마케팅비 명목으로 지급받는 방식으로 전가하고 있다. 자회사는 국내 광고선전비와 해외 본사에 지급한 국제마케팅비를 손금처리하기 때문에, 원천지국 입장에서는 세원이 잠식되고 소득이 이전되는 결과가 발생한다. 다국적기업의 광고선전비를 이용한 조세회피를 막기 위해, 자회사의 국내마케팅활동에 대해서는 마케팅 무형자산을 인정하고, 자회사가 부담하는 국제마케팅비에 대해서는 사용료소득으로 인식함으로써 원천지국의 과세권을 인정하는 방안에 관하여 논의할 필요가 있다. 세계 각국은 세수 확보를 위해 마케팅 무형자산 개념을 적극 활용하여 원천지국의 과세권을 행사하고 있고, 이러한 흐름은 2017년 개정된 OECD 이전가격지침과 UN 이전가격 매뉴얼에도 반영되었다. 우리나라도 국내세법에 마케팅 무형자산 개념을 도입하여 자회사의 마케팅활동에 대한 적정한 보상을 인정함으로써 원천지국과 거주지국 간 과세소득의 공정한 분배를 도모해야 한다. 자회사가 해외 본사에 국제마케팅비를 지급하는 경우, 소득구분에 따라 원천지국 과세권의 유무와 범위가 달라진다. 그 때문에 유통업자는 원가분담약정에 따른 분담금이나 그룹 내부 용역거래에 대한 대가로 보아 원천징수를 피하려고 하고, 과세관청은 사용료소득으로 보아 원천징수의무를 부과하려고 한다. 국제마케팅비는 해외 본사가 보유한 상표의 가치를 강화하는 국제마케팅활동에 쓰였으므로, 국제마케팅활동으로 인해 상승된 상표권의 사용대가, 즉 사용료소득으로 보는 것이 타당하다.
-----------------------------
Many multinational enterprises are burdening their subsidiaries in charge of distribution function with advertising, marketing and p

In [138]:
dict_df["src"]="korean"
dict_df["tgt"]="english"
dict_df["src"].iloc[:len(dict_df)//2]="english"
dict_df["tgt"].iloc[:len(dict_df)//2]="korean"
dict_df=dict_df.sample(frac=1)

In [153]:
dict_df_sample_100=dict_df.iloc[:100]

In [146]:
tax_law["src"]="korean"
tax_law["tgt"]="english"
tax_law["src"].iloc[:len(tax_law)//2]="english"
tax_law["tgt"].iloc[:len(tax_law)//2]="korean"
tax_law=tax_law.sample(frac=1)

/tmp/ipykernel_187/2306423426.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  tax_law["src"].iloc[:len(tax_law)//2]="english"
/tmp/ipykernel_187/2306423426.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!

In [161]:
train_df_concat=pd.concat([train_df_sample_10k,dict_df_sample_100,tax_law]).sample(frac=1).reset_index(drop=True)

In [162]:
train_dataset_10k=datasets.Dataset.from_pandas(train_df_concat)

In [238]:
train_dataset_10k[67]

{'english': 'Article XII Paragraph 1 of Article 22 (Elimination of Double Taxation) of the Convention shall be deleted and replaced by the following: “1. Subject to the provisions of Korean tax law regarding the allowance as a credit against Korean tax of tax payable in any country other than Korea (which shall not affect the general principle hereof): (a) the Swiss tax payable (excluding, in the case of a dividend, tax payable in respect of profits out of which the dividend is paid) under the laws of Switzerland and in accordance with this Convention, whether directly or by deduction, in respect of income from sources within Switzerland shall be allowed as a credit against Korean tax payable in respect of that income. The credit shall not, however, exceed that portion of Korean tax which the income from sources within Switzerland bears to the entire income subject to Korean tax. (b) In the case of a dividend paid by a company which is a resident of Switzerland to a company which is a 

In [239]:
train_dataset_10k.save_to_disk("/azurestorage/data/translation_data/alinged_dataset/prepared_for_training/training_dataset_10k/")

Saving the dataset (0/1 shards):   0%|          | 0/10819 [00:00<?, ? examples/s]

# few shot dataset

In [6]:
dastset_few=Dataset.load_from_disk("/azurestorage/data/translation_data/aligned_dataset/prepared_for_training/training_dataset_124shots/")

In [12]:
dataset_10k=Dataset.load_from_disk("/azurestorage/data/translation_data/aligned_dataset/prepared_for_training/training_dataset_10k/")

In [ ]:
dataset_10k.filter

In [13]:
set(dataset_10k["category"])

{None,
 'ele_sci',
 'etc',
 'food',
 'human_art',
 'ifrs esg',
 'pilar',
 'shatgpt',
 '경제',
 '구어체_대화체',
 '농업·수산',
 '민사법',
 '법률',
 '상업·무역·공업',
 '육운·항공·관광',
 '재경·경제일반',
 '정보통신',
 '주택·건축·도로',
 '환경·노동'}

In [52]:
filtered_dataset_10k=dataset_10k.filter(lambda x:x["category"]==None or x["category"] in ["ele_sci","etc","food","human_art","경제","shatgpt",'구어체_대화체'])

Filter:   0%|          | 0/10819 [00:00<?, ? examples/s]

In [78]:
filtered_dataset_10k=filtered_dataset_10k.filter(lambda x:"AAA" not in x["korean"])

Filter:   0%|          | 0/7185 [00:00<?, ? examples/s]

In [55]:
filtered_dataset_few=dastset_few.filter(lambda x:x["category"] in [ '법률','육운·항공·관광','재경·경제일반',"ifrs_esg","pilar"])

Filter:   0%|          | 0/124 [00:00<?, ? examples/s]

In [79]:
new_dataset_7k=concatenate_datasets([filtered_dataset_10k,filtered_dataset_few]).shuffle()

In [80]:
new_dataset_7k_df=new_dataset_7k.to_pandas()

In [85]:
new_dataset_7k_df=new_dataset_7k_df.drop_duplicates(subset=["english","korean"])

In [86]:
new_dataset_7k=Dataset.from_pandas(new_dataset_7k_df)

In [87]:
new_dataset_7k.save_to_disk("/azurestorage/data/translation_data/aligned_dataset/prepared_for_training/training_dataset_7k/")

Saving the dataset (0/1 shards):   0%|          | 0/6941 [00:00<?, ? examples/s]